In [1]:
import pandas as pd

In [2]:
df = pd.read_excel('ab_data_z_test_conversions.xlsx')

Здесь:

- user_id — идентификатор покупателя.
- timestamp — время захода на сайт.
- group — группа, в которой находится покупатель (control/treatment).
- landing_page — старая или новая версия лендинга показывалась покупателю.
- converted — совершил ли посетитель покупку.

In [3]:
df

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1
...,...,...,...,...,...
294473,751197,2017-01-03 22:28:38.630509,control,old_page,0
294474,945152,2017-01-12 00:51:57.078372,control,old_page,0
294475,734608,2017-01-22 11:45:03.439544,control,old_page,0
294476,697314,2017-01-15 01:20:28.957438,control,old_page,0


Для начала проверим уникальные эелементов в каждлй колонке

In [4]:
df.nunique()

user_id         290584
timestamp       294478
group                2
landing_page         2
converted            2
dtype: int64

Видим что в столбце timestamp у нас чуть больше уникальных значений чем в в столбце user_id. Это может говорить о том что пользователи заходили несколько раз в наш продукт

Посмотрим на эти значения

In [5]:
df[df.duplicated(['user_id'], keep=False)].sort_values('user_id')

,user_id,timestamp,group,landing_page,converted
230259,630052,2017-01-17 01:16:05.208766,treatment,new_page,0
213114,630052,2017-01-07 12:25:54.089486,treatment,old_page,1
22513,630126,2017-01-14 13:35:54.778695,treatment,old_page,0
251762,630126,2017-01-19 17:16:00.280440,treatment,new_page,0
183371,630137,2017-01-20 02:08:49.893878,control,old_page,0
...,...,...,...,...,...
142354,945703,2017-01-08 19:40:51.169351,control,new_page,0
186960,945797,2017-01-13 17:23:21.750962,control,old_page,0
40370,945797,2017-01-11 03:04:49.433736,control,new_page,1
165143,945971,2017-01-16 10:09:18.383183,control,old_page,0


Видим что сразу же первый посетитель заходил 2 раза и попал на 2 разные страницы. То есть видел одну из страниц которую видеть не должен был.

Для того чтобы понять что с такими пользователями делать, посмотрим на их долю в общем трафике.

In [6]:
duplicated_users = df[df.duplicated('user_id', keep=False)]['user_id'].nunique()
all_users = df['user_id'].nunique()

print(f'Precentage of duplicated users {duplicated_users / all_users}')

Precentage of duplicated users 0.013400600170690747


Видим что доля таких пользователей не большая. Чуть больше 1%. Ничего страшного не случиться если мы их удалим

In [7]:
df = df.drop_duplicates('user_id', keep=False)

Проверим еще раз уникальные значения в столбцах

In [8]:
df.nunique()

user_id         286690
timestamp       286690
group                2
landing_page         2
converted            2
dtype: int64

Видим что теперь с элементами все впорядке

Дополнительно проверим что исключена ситуация когда кто-то из тестовой группы не попадает на старый лендинг и наоборот

In [9]:
print(df[(df['group'] == 'treatment') & (df['landing_page'] == 'old_page')].shape[0])
print(df[(df['group'] == 'treatment') & (df['landing_page'] == 'new_page')].shape[0])
print(df[(df['group'] == 'control') & (df['landing_page'] == 'old_page')].shape[0])
print(df[(df['group'] == 'control') & (df['landing_page'] == 'new_page')].shape[0])

0
143397
143293
0


Отлично. У нас нет ситуации когда пользователь видел бы страницу которую видеть не должен был.

Теперь посмотрим на базовые статистики

In [10]:
df.groupby('group').converted.mean()

group
control      0.120173
treatment    0.122938
Name: converted, dtype: float64

Видим что в контрольной группе конверсии 0.120173 а в тестовой группе 0.122938

Для проверки значимости конверсии будем использовать статистический критерий Z-тест. Для начало импортируем ztest

In [11]:
from statsmodels.stats.weightstats import ztest

Для теста устанавливем следующие условия
- H0: нет никакой разници в конверсии между старой и новой страницы
- H1: новый лендинг дает значимо большую конверсию чем старый

За alpha возьмем стандартное значение в 0.05

In [12]:
statistics, p_val = ztest(df[df['group'] == 'treatment'].converted.values,
                         df[df['group'] == 'control'].converted.values)

print(f'p-value is: {p_val}')

p-value is: 0.023490326680407174


Как видим p-value равен 0.23, это меньше нашего уровня значимости в 0.05. Следовательно мы можем отвергнуть нашу нулевую гипотезу (H0) о том что нет ни какой разницы между старой и новой страницей.

Можно утверждать что новый лендинг работает лучше и мы это статистический доказали и можно раскатать новый лендинг на всю 100% аудиторию!